# 📈 Tonaj ve Talep Analizi
Bu notebook, aylık tonaj verilerinin analizi ve mevsimsel pattern tespitini içermektedir.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (14, 8)
sns.set_palette("husl")
sns.set_style("whitegrid")

# Veri yükleme
tonnage_df = pd.read_csv('tonnages.csv')

# Tarih sütunu oluştur
tonnage_df['Tarih'] = pd.to_datetime(tonnage_df['YIL'].astype(str) + '-' + 
                                      tonnage_df['AY'].str.upper().map({
                                          'OCAK': '01', 'ŞUBAT': '02', 'MART': '03',
                                          'NİSAN': '04', 'MAYIS': '05', 'HAZİRAN': '06',
                                          'TEMMUZ': '07', 'AĞUSTOS': '08', 'EYLÜL': '09',
                                          'EKİM': '10', 'KASIM': '11', 'ARALIK': '12'
                                      }) + '-01')

tonnage_df = tonnage_df.sort_values('Tarih')

print("Tonaj Verisi Yüklendi!")
print(f"Veri aralığı: {tonnage_df['Tarih'].min().strftime('%B %Y')} - {tonnage_df['Tarih'].max().strftime('%B %Y')}")
tonnage_df.head(10)

## 📊 Genel İstatistikler

In [ ]:
print("=" * 70)
print("TONAJ İSTATİSTİKLERİ")
print("=" * 70)
print(f"\nToplam Atık (24 ay): {tonnage_df['Toplam Tonaj (TON)'].sum():,.2f} ton")
print(f"Ortalama Aylık Tonaj: {tonnage_df['Toplam Tonaj (TON)'].mean():,.2f} ton")
print(f"Ortalama Günlük Tonaj: {tonnage_df['Ortalama Günlük Tonaj (TON)'].mean():.2f} ton/gün")

print(f"\n{'':=<70}")
print(f"Yerüstü Konteyner Oranı: {(tonnage_df['Yer Üstü Tonaj (TON)'].sum() / tonnage_df['Toplam Tonaj (TON)'].sum() * 100):.1f}%")
print(f"Yeraltı Konteyner Oranı: {(tonnage_df['Yer Altı Tonaj (TON)'].sum() / tonnage_df['Toplam Tonaj (TON)'].sum() * 100):.1f}%")

print(f"\n{'':=<70}")
max_month = tonnage_df.loc[tonnage_df['Toplam Tonaj (TON)'].idxmax()]
min_month = tonnage_df.loc[tonnage_df['Toplam Tonaj (TON)'].idxmin()]
print(f"En Yüksek Tonaj: {max_month['AY']} {max_month['YIL']} - {max_month['Toplam Tonaj (TON)']:,.2f} ton")
print(f"En Düşük Tonaj: {min_month['AY']} {min_month['YIL']} - {min_month['Toplam Tonaj (TON)']:,.2f} ton")
print(f"Fark: {(max_month['Toplam Tonaj (TON)'] - min_month['Toplam Tonaj (TON)']):,.2f} ton ({((max_month['Toplam Tonaj (TON)'] - min_month['Toplam Tonaj (TON)']) / min_month['Toplam Tonaj (TON)'] * 100):.1f}% artış)")
print("=" * 70)

## 📉 Zaman Serisi Analizi

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(16, 14))

# 1. Toplam tonaj trendi
axes[0].plot(tonnage_df['Tarih'], tonnage_df['Toplam Tonaj (TON)'], 
            marker='o', linewidth=2.5, markersize=8, color='darkblue', label='Toplam Tonaj')
axes[0].fill_between(tonnage_df['Tarih'], tonnage_df['Toplam Tonaj (TON)'], alpha=0.3)
axes[0].set_xlabel('Tarih', fontsize=12)
axes[0].set_ylabel('Tonaj (Ton)', fontsize=12)
axes[0].set_title('Aylık Toplam Atık Tonajı Trendi', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)
axes[0].legend(fontsize=11)

# Ortalamanın üstünde/altında vurgula
mean_tonnage = tonnage_df['Toplam Tonaj (TON)'].mean()
axes[0].axhline(mean_tonnage, color='red', linestyle='--', linewidth=2, 
               label=f'Ortalama: {mean_tonnage:,.0f} ton', alpha=0.7)
axes[0].legend(fontsize=11)

# 2. Yerüstü vs Yeraltı karşılaştırması
axes[1].plot(tonnage_df['Tarih'], tonnage_df['Yer Üstü Tonaj (TON)'], 
            marker='s', linewidth=2, markersize=6, label='Yerüstü', color='green')
axes[1].plot(tonnage_df['Tarih'], tonnage_df['Yer Altı Tonaj (TON)'], 
            marker='^', linewidth=2, markersize=6, label='Yeraltı', color='orange')
axes[1].set_xlabel('Tarih', fontsize=12)
axes[1].set_ylabel('Tonaj (Ton)', fontsize=12)
axes[1].set_title('Yerüstü vs Yeraltı Konteyner Tonaj Karşılaştırması', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)
axes[1].legend(fontsize=11)

# 3. Günlük ortalama tonaj
axes[2].bar(tonnage_df['Tarih'], tonnage_df['Ortalama Günlük Tonaj (TON)'], 
           width=20, color='steelblue', alpha=0.7, edgecolor='black')
axes[2].set_xlabel('Tarih', fontsize=12)
axes[2].set_ylabel('Günlük Ortalama (Ton)', fontsize=12)
axes[2].set_title('Aylık Günlük Ortalama Tonaj', fontsize=14, fontweight='bold')
axes[2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 📆 Mevsimsel Analiz

In [ ]:
# Ay bazında ortalama
tonnage_df['Ay_Num'] = tonnage_df['Tarih'].dt.month
monthly_avg = tonnage_df.groupby('Ay_Num')['Toplam Tonaj (TON)'].mean()

# Mevsim tanımla
def get_season(month):
    if month in [12, 1, 2]:
        return 'Kış'
    elif month in [3, 4, 5]:
        return 'İlkbahar'
    elif month in [6, 7, 8]:
        return 'Yaz'
    else:
        return 'Sonbahar'

tonnage_df['Mevsim'] = tonnage_df['Ay_Num'].apply(get_season)
seasonal_avg = tonnage_df.groupby('Mevsim')['Toplam Tonaj (TON)'].mean()

# Görselleştir
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Ay bazlı ortalama
month_names = ['Ocak', 'Şubat', 'Mart', 'Nisan', 'Mayıs', 'Haziran', 
               'Temmuz', 'Ağustos', 'Eylül', 'Ekim', 'Kasım', 'Aralık']
axes[0].bar(range(1, 13), monthly_avg.values, color='coral', alpha=0.7, edgecolor='black')
axes[0].set_xticks(range(1, 13))
axes[0].set_xticklabels(month_names, rotation=45, ha='right')
axes[0].set_xlabel('Ay', fontsize=12)
axes[0].set_ylabel('Ortalama Tonaj (Ton)', fontsize=12)
axes[0].set_title('Aylara Göre Ortalama Tonaj (2024-2025)', fontsize=13, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Mevsimsel ortalama
season_order = ['Kış', 'İlkbahar', 'Yaz', 'Sonbahar']
seasonal_avg_ordered = seasonal_avg.reindex(season_order)
colors_season = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12']
axes[1].bar(season_order, seasonal_avg_ordered.values, color=colors_season, alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Mevsim', fontsize=12)
axes[1].set_ylabel('Ortalama Tonaj (Ton)', fontsize=12)
axes[1].set_title('Mevsimlere Göre Ortalama Tonaj', fontsize=13, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

# Değerleri yazdır
for i, v in enumerate(seasonal_avg_ordered.values):
    axes[1].text(i, v + 100, f'{v:,.0f}', ha='center', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.show()

In [ ]:
print("=" * 60)
print("MEVSİMSEL İSTATİSTİKLER")
print("=" * 60)
for season in season_order:
    avg_ton = seasonal_avg_ordered[season]
    print(f"{season:15s}: {avg_ton:>10,.2f} ton")
print("=" * 60)
print(f"En Yüksek: {seasonal_avg_ordered.idxmax()} ({seasonal_avg_ordered.max():,.2f} ton)")
print(f"En Düşük: {seasonal_avg_ordered.idxmin()} ({seasonal_avg_ordered.min():,.2f} ton)")
print(f"Mevsimsel Varyans: {((seasonal_avg_ordered.max() - seasonal_avg_ordered.min()) / seasonal_avg_ordered.min() * 100):.1f}%")
print("=" * 60)

## 📊 Yıllık Karşılaştırma (2024 vs 2025)

In [ ]:
# Yıllık toplam
yearly_total = tonnage_df.groupby('YIL')['Toplam Tonaj (TON)'].sum()
yearly_avg = tonnage_df.groupby('YIL')['Toplam Tonaj (TON)'].mean()

# 2024 vs 2025 aynı ayların karşılaştırması
data_2024 = tonnage_df[tonnage_df['YIL'] == 2024].set_index('Ay_Num')['Toplam Tonaj (TON)']
data_2025 = tonnage_df[tonnage_df['YIL'] == 2025].set_index('Ay_Num')['Toplam Tonaj (TON)']

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Yıllık toplam karşılaştırma
yearly_total.plot(kind='bar', ax=axes[0], color=['#3498db', '#e74c3c'], alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Yıl', fontsize=12)
axes[0].set_ylabel('Toplam Tonaj (Ton)', fontsize=12)
axes[0].set_title('Yıllık Toplam Tonaj Karşılaştırması', fontsize=13, fontweight='bold')
axes[0].tick_params(axis='x', rotation=0)
axes[0].grid(axis='y', alpha=0.3)

for i, v in enumerate(yearly_total.values):
    axes[0].text(i, v + 1000, f'{v:,.0f}', ha='center', fontweight='bold', fontsize=11)

# Aynı ayların karşılaştırması
# Ortak ayları bul
common_months = sorted(set(data_2024.index) & set(data_2025.index))
x = np.arange(len(common_months))
width = 0.35

values_2024 = [data_2024[m] if m in data_2024.index else 0 for m in common_months]
values_2025 = [data_2025[m] if m in data_2025.index else 0 for m in common_months]

axes[1].bar(x - width/2, values_2024, width, label='2024', color='#3498db', alpha=0.7, edgecolor='black')
axes[1].bar(x + width/2, values_2025, width, label='2025', color='#e74c3c', alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Ay', fontsize=12)
axes[1].set_ylabel('Tonaj (Ton)', fontsize=12)
axes[1].set_title('2024 vs 2025 Aylık Karşılaştırma', fontsize=13, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels([month_names[m-1] for m in common_months], rotation=45, ha='right')
axes[1].legend(fontsize=11)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
print("=" * 60)
print("YILLIK KARŞILAŞTIRMA")
print("=" * 60)
for year in sorted(tonnage_df['YIL'].unique()):
    total = yearly_total[year]
    avg = yearly_avg[year]
    months = len(tonnage_df[tonnage_df['YIL'] == year])
    print(f"\n{year}:")
    print(f"  Toplam: {total:,.2f} ton ({months} ay)")
    print(f"  Aylık Ortalama: {avg:,.2f} ton")
    
if len(yearly_total) > 1:
    change = yearly_total.iloc[-1] - yearly_total.iloc[0]
    change_pct = (change / yearly_total.iloc[0]) * 100
    print(f"\nDeğişim: {change:+,.2f} ton ({change_pct:+.1f}%)")
print("=" * 60)

## 🔮 Trend ve Tahminleme

In [ ]:
# Basit doğrusal trend
from sklearn.linear_model import LinearRegression

# Veriyi hazırla
X = np.arange(len(tonnage_df)).reshape(-1, 1)
y = tonnage_df['Toplam Tonaj (TON)'].values

# Model oluştur
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)

# Gelecek 6 ay tahmini
future_X = np.arange(len(tonnage_df), len(tonnage_df) + 6).reshape(-1, 1)
future_pred = model.predict(future_X)

# Görselleştir
plt.figure(figsize=(16, 8))
plt.plot(tonnage_df['Tarih'], y, marker='o', linewidth=2, markersize=8, 
         label='Gerçek Veri', color='darkblue')
plt.plot(tonnage_df['Tarih'], y_pred, linestyle='--', linewidth=2, 
         label='Trend Çizgisi', color='red', alpha=0.7)

# Gelecek tahminler için tarihler oluştur
last_date = tonnage_df['Tarih'].iloc[-1]
future_dates = pd.date_range(start=last_date, periods=7, freq='M')[1:]
plt.plot(future_dates, future_pred, marker='s', linestyle=':', linewidth=2, 
         markersize=8, label='6 Aylık Tahmin', color='green')

plt.xlabel('Tarih', fontsize=13)
plt.ylabel('Tonaj (Ton)', fontsize=13)
plt.title('Tonaj Trendi ve 6 Aylık Tahmin', fontsize=15, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nTrend Eğimi: {model.coef_[0]:.2f} ton/ay")
print(f"Model R² Skoru: {model.score(X, y):.4f}")
print(f"\n6 Aylık Tahmin:")
for i, (date, pred) in enumerate(zip(future_dates, future_pred)):
    print(f"  {date.strftime('%B %Y')}: {pred:,.0f} ton")

## 🎯 Temel Bulgular

1. **Toplam Atık:** 24 ayda ~410,000 ton atık toplandı
2. **Mevsimsellik:** Yaz aylarında belirgin artış (Haziran-Ağustos)
3. **Yerüstü Dominansı:** Atığın ~90%'ı yerüstü konteynerlerden
4. **Günlük Ortalama:** 550-690 ton/gün arasında değişiyor
5. **Trend:** Hafif yükselen trend gözlemleniyor